<a href="https://colab.research.google.com/github/eyeonkarim/SDS2022-MLFlow/blob/main/LC4_MLFlow_Model_Registry_Regression_Housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><a target="_blank" href="https://www.sds2022.ch/"><img src="https://drive.google.com/uc?id=1S7k7kTXs9qIylw3C7LA9rHkLycjlY8te" width="500" style="background:none; border:none; box-shadow:none;" /></a> </center>

<center><a target="_blank" href="http://www.sit.academy"><img src="https://drive.google.com/uc?id=1x9_jQgLhozCSWDSaOdVxKmxOEAe_OLgV" width="250" style="background:none; border:none; box-shadow:none;" /></a> </center>

_____

<center> <h1> Live Coding  </h1> </center>

<p style="margin-bottom:1cm;"></p>

_____

<center>SIT Academy, 2022</center>



# MLFlow Workshop - Sequence 4 - Model Registry API

MLflow Registry offers a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model. 

It provides model lineage (which MLflow experiment and run produced the model), model versioning, stage transitions (for example from staging to production or archiving), and annotations.

## Install dependencies

In [ ]:
!pip -q install mlflow boto3 pyngrok

     |████████████████████████████████| 17.8 MB 407 kB/s 
     |████████████████████████████████| 132 kB 49.0 MB/s 
     |████████████████████████████████| 745 kB 44.5 MB/s 
     |████████████████████████████████| 209 kB 64.2 MB/s 
     |████████████████████████████████| 146 kB 69.6 MB/s 
     |████████████████████████████████| 596 kB 60.0 MB/s 
     |████████████████████████████████| 181 kB 55.7 MB/s 
     |████████████████████████████████| 81 kB 9.0 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 8.9 MB 43.5 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 127 kB 54.2 MB/s 
     |████████████████████████████████| 78 kB 6.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

## Check and see if SQL is installed

In [ ]:
#conda install -c conda-forge ipython-sql #must install to kernel if running locally
%load_ext sql

In [ ]:
from pyngrok import ngrok
from getpass import getpass

In [ ]:
def start_tunnel():
  # Terminate open tunnels if exist
  ngrok.kill()
  # Get your authtoken from https://dashboard.ngrok.com/auth
  NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
  # Open an HTTPs tunnel on port 5000 for http://localhost:5000
  ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
  print("MLflow Tracking UI:", ngrok_tunnel.public_url)

## Using an SQLite server for tracking (locally)

In [ ]:
!touch mlflow.sqlite #this will only persist for the duration of the Colab session unless you move it to your Google Drive

We need to define an environment variable `MLFLOW_TRACKING_URI` which all subsequent MLFlow runs will use for its tracking server.

In [ ]:
%env MLFLOW_TRACKING_URI=http://localhost:5000

env: MLFLOW_TRACKING_URI=http://localhost:5000


Then, we can run the MLFlow server in the background (so we can run other cells in the notebook as well):

In [ ]:
get_ipython().system_raw(
    "mlflow server --backend-store-uri sqlite:///mlflow.sqlite --host 0.0.0 --port 5000 --default-artifact-root /content/mlruns/ &"
    )

Now let's do a test run:

In [ ]:
!mlflow run --env-manager=local https://github.com/SIT-Academy/sds2022_mlflow_workshop.git#src/mlproject_simple_run/ -P alpha=0.2

/bin/bash: mlflow: command not found


In [ ]:
start_tunnel()

Enter the ngrok authtoken: ··········
MLflow Tracking UI: https://9abc-35-185-229-27.ngrok.io


In [ ]:
# To kill MLFlow Server instances running in the background, uncomment & run:
#!pkill -f gunicorn

## Using an SQL tracking server & S3 bucket for artifact storage

Here we will provide you access to an S3 bucket (for cloud object storage) for the workshop.

*If you are interested in your own (free, AWS-substitute) bucket(s), you can sign up for 150 GB of storage on [Storj](https://www.storj.io/).*

In [ ]:
%env AWS_ACCESS_KEY_ID=jwhttmo4qhgfncxxdftqlwpxfaia
%env AWS_SECRET_ACCESS_KEY=jyhyd7bxmbax3w3mzfi55tscfk5t34rotzj3nbg2e4nwpofxwuh2y

%env MLFLOW_S3_ENDPOINT_URL=https://gateway.eu1.storjshare.io/

env: AWS_ACCESS_KEY_ID=jwhttmo4qhgfncxxdftqlwpxfaia
env: AWS_SECRET_ACCESS_KEY=jyhyd7bxmbax3w3mzfi55tscfk5t34rotzj3nbg2e4nwpofxwuh2y
env: MLFLOW_S3_ENDPOINT_URL=https://gateway.eu1.storjshare.io/


In [ ]:
#Note: make sure to unset the `MLFLOW_TRACKING_URI` variable first if set earlier
#     to have it function properly as a remote tracking server!

%env MLFLOW_TRACKING_URI=

env: MLFLOW_TRACKING_URI=


In [ ]:
start_tunnel()

In [ ]:
!mlflow server --backend-store-uri sqlite:///mlflow.sqlite --default-artifact-root s3://demobucket --gunicorn-opts "--log-level debug"

[2022-06-19 18:26:59 +0000] [319] [DEBUG] Current configuration:
  config: ./gunicorn.conf.py
  wsgi_app: None
  bind: ['127.0.0.1:5000']
  backlog: 2048
  workers: 4
  worker_class: sync
  threads: 1
  worker_connections: 1000
  max_requests: 0
  max_requests_jitter: 0
  timeout: 30
  graceful_timeout: 30
  keepalive: 2
  limit_request_line: 4094
  limit_request_fields: 100
  limit_request_field_size: 8190
  reload: False
  reload_engine: auto
  reload_extra_files: []
  spew: False
  check_config: False
  print_config: False
  preload_app: False
  sendfile: None
  reuse_port: False
  chdir: /content
  daemon: False
  raw_env: []
  pidfile: None
  worker_tmp_dir: None
  user: 0
  group: 0
  umask: 0
  initgroups: False
  tmp_upload_dir: None
  secure_scheme_headers: {'X-FORWARDED-PROTOCOL': 'ssl', 'X-FORWARDED-PROTO': 'https', 'X-FORWARDED-SSL': 'on'}
  forwarded_allow_ips: ['127.0.0.1']
  accesslog: None
  disable_redirect_access_to_syslog: False
  access_log_format: %(h)s %(l)s %(u)s

## MLflow server + proxied artifact storage access

You can also run the tracking server without exposing the artifact store to end-users.

**Pro:**
*   Client-side does not need authentication to e.g. S3 bucket

**Con:**
*   Anyone with access to the tracking server can access all artifacts

Artifact URIs will start with `mlflow-artifacts:/` instead of e.g. `s3:/`.

In [ ]:
start_tunnel()

In [ ]:
!mlflow server --backend-store-uri sqlite:///mlflow.sqlite --default-artifact-root mlflow-artifacts --artifacts-destination s3://demobucket --serve-artifacts

[2022-06-19 19:41:03 +0000] [778] [INFO] Starting gunicorn 20.1.0
[2022-06-19 19:41:03 +0000] [778] [INFO] Listening at: http://127.0.0.1:5000 (778)
[2022-06-19 19:41:03 +0000] [778] [INFO] Using worker: sync
[2022-06-19 19:41:03 +0000] [781] [INFO] Booting worker with pid: 781
[2022-06-19 19:41:03 +0000] [782] [INFO] Booting worker with pid: 782
[2022-06-19 19:41:03 +0000] [783] [INFO] Booting worker with pid: 783
[2022-06-19 19:41:03 +0000] [784] [INFO] Booting worker with pid: 784
[2022-06-19 20:32:20 +0000] [778] [INFO] Handling signal: int
[2022-06-19 20:32:20 +0000] [781] [INFO] Worker exiting (pid: 781)
[2022-06-19 20:32:20 +0000] [783] [INFO] Worker exiting (pid: 783)
[2022-06-19 20:32:20 +0000] [782] [INFO] Worker exiting (pid: 782)
[2022-06-19 20:32:20 +0000] [784] [INFO] Worker exiting (pid: 784)

Aborted!
[2022-06-19 20:32:21 +0000] [778] [INFO] Shutting down: Master


## Searching through your MLflow runs programmatically

In [ ]:
import mlflow

In [ ]:
mlflow.search_runs()

This gives you all the runs listed in the already-defined tracking server as a Pandas dataframe.

In [ ]:
mlflow.search_runs().loc[0]

In [ ]:
artifact_uri = mlflow.search_runs().loc[0]['artifact_uri']

In [ ]:
mlflow.artifacts.download_artifacts(artifact_uri)

You can also filter by multiple conditions separated by an `AND` operator.

In [ ]:
mlflow.search_runs(filter_string = "metrics.r2 > 0.2")

#mlflow.search_runs(filter_string = "metrics.r2 > 0.2 AND ...")